In [ ]:
def read_graph_with_isolated_nodes(file_path):
    graph = {}
    all_nodes = set()  # 用于记录所有出现过的节点
    with open(file_path, 'r') as file:
        for line in file:
            # 跳过注释行（# 或其他符号）
            if line.startswith('#'):
                continue
            
            # 读取边的起点和终点
            source, target = map(int, line.strip().split())
            
            # 记录所有节点
            all_nodes.update([source, target])
            
            # 构建邻接列表（有向图逻辑）
            if source not in graph:
                graph[source] = []
            if target not in graph[source]:  # 避免重复边
                graph[source].append(target)
    
    # 添加孤立节点（没有出边的节点）
    for node in all_nodes:
        if node not in graph:
            graph[node] = []
    
    return graph

def print_graph_info(graph):
    # 节点数：邻接表中的所有键值对数量
    num_nodes = len(graph)
    
    # 边数：计算所有出边的数量总和
    num_edges = sum(len(targets) for targets in graph.values())
    
    print(f"Number of nodes: {num_nodes}")
    print(f"Number of edges: {num_edges}")


file_path = '/Users/shuzhou/Downloads/cit-Patents.txt'
graph = read_graph_with_isolated_nodes(file_path)

# 打印节点和边数
print_graph_info(graph)


Number of nodes: 3774768
Number of edges: 16518948


In [29]:
def degree_discount_ic(graph, k, p):
    """
    使用度折扣启发式算法选择种子节点以最大化影响力传播。

    参数:
    - graph: dict，表示图的邻接列表，key 是节点，value 是该节点的邻居列表。
    - k: int，要选择的种子节点数量。
    - p: float，传播概率。

    返回:
    - seeds: list，包含选出的种子节点。
    """
    # 初始化
    degree = {v: len(neighbors) for v, neighbors in graph.items()}  # 每个节点的度
    t = {v: 0 for v in graph}  # 每个节点已选为种子的邻居数
    dd = degree.copy()  # 初始化度折扣值
    seeds = []  # 存放种子节点
    
    for _ in range(k):
        # 选择度折扣值最大的节点作为新种子
        u = max(dd, key=dd.get)
        seeds.append(u)
        
        # 更新新种子节点的邻居的度折扣值
        for v in graph[u]:
            if v not in seeds:  # 如果邻居节点尚未被选为种子
                t[v] += 1  # 增加该节点的已选种子邻居数
                dd[v] = degree[v] - 2 * t[v] - (degree[v] - t[v]) * t[v] * p  # 更新度折扣值
        
        # 移除已经选为种子的节点的度折扣值
        dd.pop(u)
    
    return seeds


# 参数
k = 50  
p = 0.01  # 传播概率

seeds = degree_discount_ic(graph, k, p)
print("选择的种子节点:", seeds)


选择的种子节点: [5795784, 5887243, 5856194, 5855655, 5891229, 5908495, 5858586, 6008268, 5885337, 5837429, 5786132, 5782963, 5709955, 5773182, 5865471, 5733693, 5681380, 5721287, 5901425, 5817744, 5700850, 5618907, 5739256, 5587105, 5602226, 5573854, 5554769, 5969079, 5994152, 5705574, 5683843, 5643701, 5616443, 5747550, 5739175, 5798015, 5811199, 5849411, 5714566, 5756597, 5643356, 5645964, 5610317, 5989237, 5998220, 5830548, 5957898, 5877028, 5738921, 5797877]


In [28]:
import heapq

def degree_discount_ic_heap(graph, k, p):
    """
    使用度折扣启发式算法，并利用堆优化，选择种子节点以最大化影响力传播。

    参数:
    - graph: dict，图的邻接列表，key 是节点，value 是邻居列表。
    - k: int，要选择的种子节点数量。
    - p: float，传播概率。

    返回:
    - seeds: list，包含选出的种子节点。
    """
    # 初始化
    degree = {v: len(neighbors) for v, neighbors in graph.items()}  # 每个节点的度
    t = {v: 0 for v in graph}  # 每个节点已选为种子的邻居数
    dd = degree.copy()  # 度折扣值
    heap = []  # 使用最小堆（存储负值模拟最大堆）
    seeds = []  # 最终的种子节点集合
    
    # 构建初始堆
    for v, val in dd.items():
        heapq.heappush(heap, (-val, v))  # 堆存储 (-度折扣值, 节点)
    
    # 贪婪选择种子节点
    for _ in range(k):
        while True:
            # 弹出堆顶，获取最大度折扣值节点
            _, u = heapq.heappop(heap)
            # 检查是否是最新的度折扣值
            if -dd[u] == _:
                break  # 最新的值，继续执行
            # 否则跳过，因为它是旧值
        
        # 添加种子节点
        seeds.append(u)
        
        # 更新邻居节点的度折扣值
        for v in graph[u]:
            if v not in seeds:  # 如果邻居节点尚未被选为种子
                t[v] += 1  # 更新该节点已选种子邻居数
                # 计算新的度折扣值
                dd[v] = degree[v] - 2 * t[v] - (degree[v] - t[v]) * t[v] * p
                # 将更新后的节点重新加入堆
                heapq.heappush(heap, (-dd[v], v))  # 存储负值
    
    return seeds
# 参数
k = 50  
p = 0.01  # 传播概率

seeds = degree_discount_ic_heap(graph, k, p)
print("选择的种子节点:", seeds)

选择的种子节点: [5795784, 5887243, 5856194, 5855655, 5891229, 5908495, 5858586, 6008268, 5885337, 5837429, 5786132, 5782963, 5709955, 5773182, 5865471, 5733693, 5681380, 5721287, 5901425, 5817744, 5700850, 5618907, 5739256, 5587105, 5602226, 5573854, 5554769, 5969079, 5994152, 5705574, 5683843, 5643701, 5616443, 5747550, 5739175, 5798015, 5811199, 5849411, 5714566, 5756597, 5643356, 5645964, 5610317, 5989237, 5998220, 5830548, 5957898, 5877028, 5738921, 5797877]


In [ ]:
import heapq

def degree_discount_lt(graph, k, alpha=0.5):
    """
    基于线性阈值模型的度折扣启发式算法。
    
    参数:
    - graph: dict，表示图的邻接列表，key 是节点，value 是该节点的邻居列表。
    - k: int，要选择的种子节点数量。
    - alpha: float，折扣调节因子，默认值为 0.5。
    
    返回:
    - seeds: list，包含选出的种子节点。
    """
    # # 初始化
    # degree = {v: len(neighbors) for v, neighbors in graph.items()}  # 每个节点的度
    # t = {v: 0 for v in graph}  # 每个节点的已选邻居数
    # dd = degree.copy()  # 初始化度折扣值
    # heap = [(-dd[v], v) for v in graph]  # 最大堆，存放 (负折扣值, 节点)
    # heapq.heapify(heap)  # 构建堆
    # seeds = []  # 存放种子节点
    # selected = set()  # 用于快速检查是否选中节点

    # for _ in range(k):
    #     # 从堆中提取当前度折扣值最大的节点
    #     while heap:
    #         neg_dd, u = heapq.heappop(heap)  # 提取堆顶元素
    #         if u not in selected:  # 确保节点未被选中
    #             break
    #     seeds.append(u)
    #     selected.add(u)
        
    #     # 更新新种子节点的邻居的度折扣值
    #     for v in graph[u]:
    #         if v not in selected:  # 如果邻居节点尚未被选为种子
    #             t[v] += 1  # 增加该节点的已选邻居数
    #             # 更新度折扣值
    #             dd[v] = degree[v] - alpha * t[v]
    #             # 将更新后的节点重新加入堆
    #             heapq.heappush(heap, (-dd[v], v))
    
    return seeds

k = 50  
seeds = degree_discount_lt(graph, k, p)
print("选择的种子节点:", seeds)

选择的种子节点: [5795784, 5887243, 5856194, 5855655, 5891229, 5908495, 5858586, 6008268, 5885337, 5837429, 5786132, 5782963, 5709955, 5773182, 5865471, 5681380, 5733693, 5721287, 5901425, 5817744, 5700850, 5618907, 5739256, 5587105, 5602226, 5573854, 5554769, 5969079, 5994152, 5705574, 5683843, 5643701, 5616443, 5747550, 5739175, 5798015, 5811199, 5849411, 5714566, 5756597, 5610317, 5643356, 5645964, 5989237, 5998220, 5830548, 5957898, 5877028, 5738921, 5797877]
